In [9]:
import pandas as pd
import numpy as np
import glob
import plotly.graph_objects as go

In [10]:
# functions written by Mitali and Girish in 2024

def theta_phi(x1, y1, z1, x2, y2, z2, x3, y3, z3):
    """
    Calculate the angles theta and phi between three points

    Parameters:
        x1,y1,z1 (float) : The coordinates of point 1.
        x2,y2,z2 (float) : The coordinates of point 2.
        x3,y3,z3 (float) : The coordinates of point 3.

    Returns:
        theta (float) : The angle (in degrees) between point1, point 2 and point3 in the XY plane.
        phi (float) : The angle (in degrees) between point1, point 2 and point3 in the ZY plane.

    Dependencies:
        - numpy (np): Required for numerical computations and array operations.
    """

    # Convert the input coordinates to NumPy arrays for vector operations
    a = np.array([x1,y1,z1])
    b = np.array([x2,y2,z2])
    c = np.array([x3,y3,z3])

    # Calculate the vectors
    ab = b - a # changed this from ba = a -b for consistency 
    bc = c - b

    # Calculate the theta angle (angle in XY plane)
    theta = np.degrees(np.arctan2(np.cross(ab[:2], bc[:2]), np.dot(ab[:2], bc[:2])))

    # Calculate the phi angle (angle in ZY plane) 
    phi = np.degrees(np.arctan2(np.cross(ab[[0, 2]], bc[[0, 2]]), np.dot(ab[[0, 2]], bc[[0, 2]])))

    return theta, phi

In [11]:
files = glob.glob('./Data/Outputs/Sampled_1k/COM/*_smoothened*_COM.csv')
files

['./Data/Outputs/Sampled_1k/COM\\2023-04-19_Trial7_Set1_5k_smoothened_interpol_COM.csv',
 './Data/Outputs/Sampled_1k/COM\\2023-04-21_Trial6_1k_smoothened_interpol_COM.csv',
 './Data/Outputs/Sampled_1k/COM\\2023-04-21_Trial7_1k_smoothened_interpol_COM.csv',
 './Data/Outputs/Sampled_1k/COM\\2023-04-28_Trial4_5k_smoothened_interpol_COM.csv',
 './Data/Outputs/Sampled_1k/COM\\2023-04-28_Trial5_5k_smoothened_interpol_COM.csv',
 './Data/Outputs/Sampled_1k/COM\\2023-04-28_Trial8_1k_smoothened_interpol_COM.csv']

In [7]:
# info = pd.read_csv('./Data/Outputs/Raw_xyz_points/metadata.csv')

In [8]:
# for _,row in info.iterrows():
#     print(row['filename'])
#     if row['filename'][:-10] in f.split('\\')[-1]:
#         print(row['framerate'])

In [108]:
for f in files:
    name = f.split('\\')[-1][:-4]
    df = pd.read_csv(f)
    
    chasee_COM = df.loc[:, ['chasee_com_x', 'chasee_com_y', 'chasee_com_z']]
    chaser_COM = df.loc[:, ['chaser_com_x', 'chaser_com_y', 'chaser_com_z']]
    
    print(chasee_COM.isnull().values.any())
    print(chaser_COM.isnull().values.any())
    
    framerate = 1000 # for subsampled data at 1k    
#     framerate = [row['framerate'] for ind, row in info.iterrows() if row['filename'][:-10] in f.split('\\')[-1]][0] 
    
    #compute linear distance, velocity and acceleration
    dist = []
    vel = []
    acc = []
    for fly in [chasee_COM, chaser_COM]:
        d = np.linalg.norm(fly, axis = 1)
        v = np.diff(d, n = 1)*framerate
        a = np.diff(d, n = 2)*framerate
        
        dist.append(d)
        vel.append(v)
        acc.append(a)

    
    linear_speed_df = pd.DataFrame(
        {'chasee_dist': dist[0],
         'chasee_vel': [np.nan] + list(vel[0]),
         'chasee_acc': [np.nan]*2 + list(acc[0]),
         'chaser_dist': dist[1],
         'chaser_vel': [np.nan] + list(vel[1]),
         'chaser_acc': [np.nan]*2 + list(acc[1]),
         })
    
    linear_speed_df.to_csv('./Data/Outputs/Sampled_1k/Speeds/' + name + '_LinearSpeedAcc.csv')
    
    #compute angular distance, velocity and acceleration
    chasee_theta = []
    chasee_phi = []
    for ind, _ in chasee_COM[:-2].iterrows():
        pt1 = chasee_COM.loc[ind, :]
        pt2 = chasee_COM.loc[ind+1, :]
        pt3 = chasee_COM.loc[ind+2, :]
        tt, ph = theta_phi(pt1['chasee_com_x'], pt1['chasee_com_y'], pt1['chasee_com_z']
                           , pt2['chasee_com_x'], pt2['chasee_com_y'], pt2['chasee_com_z']
                           , pt3['chasee_com_x'], pt3['chasee_com_y'], pt3['chasee_com_z'])
        
           
        chasee_theta.append(tt)
        chasee_phi.append(ph)
        
    chaser_theta = []
    chaser_phi = []
    for ind, _ in chaser_COM[:-2].iterrows():
        pt1 = chaser_COM.loc[ind, :]
        pt2 = chaser_COM.loc[ind+1, :]
        pt3 = chaser_COM.loc[ind+2, :]
        tt, ph = theta_phi(pt1['chaser_com_x'], pt1['chaser_com_y'], pt1['chaser_com_z']
                           , pt2['chaser_com_x'], pt2['chaser_com_y'], pt2['chaser_com_z']
                           , pt3['chaser_com_x'], pt3['chaser_com_y'], pt3['chaser_com_z'])
        
           
        chaser_theta.append(tt)
        chaser_phi.append(ph)
    

    #Calculating angular velocity of Chasee and chaser
    angvel_chasee_theta = np.diff(chasee_theta)*framerate
    angvel_chasee_phi = np.diff(chasee_phi)*framerate
    angvel_chasee_theta = np.append(angvel_chasee_theta, np.repeat(np.nan, 1))
    angvel_chasee_phi = np.append(angvel_chasee_phi, np.repeat(np.nan, 1))
    
    angvel_chaser_theta = np.diff(chaser_theta)*framerate
    angvel_chaser_phi = np. diff(chaser_phi)*framerate
    angvel_chaser_theta = np.append(angvel_chaser_theta, np.repeat(np.nan, 1))
    angvel_chaser_phi = np.append(angvel_chaser_phi, np.repeat(np.nan, 1))
    
    #Calculating angular accerelation of Chasee and chaser
    angacc_chasee_theta = np.diff(angvel_chasee_theta)
    angacc_chasee_phi = np. diff(angvel_chasee_phi)
    angacc_chasee_theta = np.append(angacc_chasee_theta, np.repeat(np.nan, 1))
    angacc_chasee_phi = np.append(angacc_chasee_phi, np.repeat(np.nan, 1))
    
    angacc_chaser_theta = np.diff(angvel_chaser_theta)*framerate
    angacc_chaser_phi = np. diff(angvel_chaser_phi)*framerate
    angacc_chaser_theta = np.append(angacc_chaser_theta, np.repeat(np.nan, 1))
    angacc_chaser_phi = np.append(angacc_chaser_phi, np.repeat(np.nan, 1))
    
    
    angular_speed_df = pd.DataFrame({'chasee_theta': chasee_theta,
                            'chasee_phi': chasee_phi,
                            'chaser_theta':chaser_theta,
                            'chaser_phi':chaser_phi,
                            'chasee_theta_speed': angvel_chasee_theta,
                             'chasee_phi_speed': angvel_chasee_phi,
                             'chaser_theta_speed': angvel_chaser_theta,
                             'chaser_phi_speed': angvel_chaser_phi,
                             'chasee_theta_acc': angacc_chasee_theta,
                             'chasee_phi_acc': angacc_chasee_phi,
                             'chaser_theta_acc': angacc_chaser_theta,
                             'chaser_phi_acc': angacc_chaser_phi})
    
    angular_speed_df.to_csv('./Data/Outputs/Sampled_1k/Speeds/' + name + '_AnglularSpeedAcc.csv')

False
False
False
False
False
False
False
False
False
False
False
False
